In [87]:
import numpy as np, matplotlib.pyplot as plt, matplotlib.gridspec as gs
from sklearn.model_selection import train_test_split,cross_validate
from sklearn.linear_model import LinearRegression
import pandas as pd
import csv,os

In [89]:
# Parametres pour estimer l'orientation
pi=np.pi
H_set_o=np.r_[0.01,0.25,0.35,0.45,0.5,0.65,0.75,0.85,0.99]
delta_set_o=np.r_[pi/64,9*pi/128,pi/8,3*pi/16,pi/4,5*pi/16,3*pi/8,7*pi/16,pi/2]
alpha_set_o=np.r_[-pi/2,-pi/6,0,pi/4,pi/3]
alpha_set_str_o=np.array(['-$\\frac{\pi}{2}$','-$\\frac{\pi}{6}$','0','$\\frac{\pi}{4}$','$\\frac{\pi}{3}$'],str)
delta_set_str_o=np.array(['$\\frac{\pi}{64}$','$\\frac{9\pi}{128}$','$\\frac{\pi}{8}$',
'$\\frac{3\pi}{16}$','$\\frac{\pi}{4}$','$\\frac{5\pi}{16}$','$\\frac{3\pi}{8}$','$\\frac{7\pi}{16}$','$\\frac{\pi}{2}$'],str)


# Parametres pour estimer H et delta
theta1=pi/2
H_set=np.r_[0.01,0.15,0.25,0.35,0.45,0.5,0.65,0.75,0.85,0.99]
delta_set=np.r_[pi/64,9*pi/128,pi/8,3*pi/16,pi/4,5*pi/16,3*pi/8,7*pi/16,pi/2]

delta_set_str=np.array(['$\\frac{\pi}{64}$','$\\frac{9\pi}{128}$','$\\frac{\pi}{8}$',
'$\\frac{3\pi}{16}$','$\\frac{\pi}{4}$','$\\frac{5\pi}{16}$','$\\frac{3\pi}{8}$','$\\frac{7\pi}{16}$','$\\frac{\pi}{2}$'],str)


# Chargement du champ X
fil_dir='img_revisions_Detec_img_param_by_Riesz.Copie/rslta_/'
#img_names=np.load(fil_dir_100+'img_names.npy',allow_pickle=True)


def load_img_200_of(H,delta,index,alpha=None):
    '''
    H: Rugosite du champ
    delta: Demi-ouverture du cone frequentiel
    index: Numero d'un realisation du champ'''


    if alpha!=None:# estimation de alpha
        name='H='+str(H)+'_delta='+str(delta)+'_alpha='+str(alpha)+'_nb='+str(index)
        title='H='+str(H)+'_delta='+str(np.round(delta,2))+'_alpha='+str(np.round(alpha,2))
        fil_dir_200='img_revisions_Detec_img_param_by_Riesz.Copie/diff_200_realisations_of_each_EF_H_alpha_delta/'
    else:# estimation de H et delta
        name='H='+str(H)+'_delta='+str(delta)+'_alpha='+str(delta)+'_nb='+str(index)
        title='H='+str(H)+'_delta='+str(np.round(delta,2))+'_alpha='+str(np.round(delta,2))
        fil_dir_200='img_revisions_Detec_img_param_by_Riesz.Copie/diff_200_realisations_of_each_EF_H_delta/'

    img_gray=np.load(fil_dir_200+name +'.npy')
    return img_gray,title

# Taille des image
img_shape=(256,256)

### Estimateur des paramètres

In [90]:
def TSORE(H,delta,alpha=None,index=0,l=3,c_n=0.014,rate=75):
    ################################################### Chargement de l'image #########################################################################
    X,_=load_img_200_of(H,delta,index,alpha)
    ################################################### Construction de la transformée de Fourier de l'ondelette hat_psi ##################################
    M,N=X.shape
    hat_psi=np.zeros((M,N))
    # Calcul des normes des arguments
    xx,yy=np.meshgrid(np.arange(-M//2,M//2),range(-N//2,N//2))  
    arg_norm=np.sqrt(xx**2+yy**2)
    coord_norm=np.fft.ifftshift(arg_norm)

    # Calcul de la transformée de fouriée de psi
    x=coord_norm*c_n
    t=(x<=pi)*(x>pi/4)
    hat_psi[t]=np.cos((pi/2)*np.log2(2*np.abs(x[t])/pi))


    ######################## Construction de la transformée de Fourier de Riesz #######################################
    hat_X_ctred=np.fft.fftshift(np.fft.fft2(X))
    xy_0=np.argwhere(arg_norm==0)[0]                                            # Prevoir l'indetermination a l'origne avant l'inversion
    arg_norm[xy_0[0],xy_0[1]]=1
    inv_arg_norm=1/arg_norm
    inv_arg_norm[[xy_0[0],xy_0[1]]]=0                                           # Prevoir l'indetermination a l'origne apres l'inversion

    hat_R1=np.fft.ifftshift(-1j*xx*inv_arg_norm*hat_X_ctred)                    # Coordonnées de Riesz 1
    hat_R2=np.fft.ifftshift(-1j*yy*inv_arg_norm*hat_X_ctred)                    # Coordonnées de Riesz 1

    ################################### Construction des Pas de translation de l'ondelette ###############################
    k1,k2=np.meshgrid(range(M),range(N))                                        
    pas=k1%M+(k2%N)*M                                                           
    pas_hat_psi_lk=((2**l)*pas)%(N*M)

    ################################### Construction des coefficients d'ondelette de Riesz ################################
    hat_coef_1=-(2**l)*hat_R1.flatten()[pas]*hat_psi.flatten()[pas_hat_psi_lk]
    hat_coef_2=-(2**l)*hat_R2.flatten()[pas]*hat_psi.flatten()[pas_hat_psi_lk]
    coef_0=np.fft.ifft2(hat_coef_1.reshape(M,N)).real
    coef_1=np.fft.ifft2(hat_coef_2.reshape(M,N)).real

    ################################### Construction du tenseur de structure de Riesz empirique ###########################
    cr0=coef_0[rate:-rate,rate:-rate];cr1=coef_1[rate:-rate,rate:-rate]
    J_00=np.sum(cr0**2)
    J_01=J_10=np.sum(cr0*cr1)
    J_11=np.sum(cr1**2)
    tsore=np.array([[J_00,J_01],[J_10,J_11]])/(M*N)

    return tsore

In [91]:
def H_delta_alpha_estimator(H,delta,alpha=None,nb_realst=25,nb_scale=5,cv=3,c_n=0.014,best_scale_delta=3):
    meanof_tild_H=0;meanof_tild_ic=0;meanof_tild=0
    scales=np.arange(1,nb_scale+1,dtype=int)
    log2_sumvalp_ic=np.empty((len(scales),2))      # (log2 de la somme des valeurs propres, indice de cohérence) par echelle.

    for index in range(nb_realst):
    ############################################################ Estimation de l'orientatio #############################################################   
        if alpha!=None:
                vals_p,vect_p=np.linalg.eig(TSORE(H,delta,alpha,index,best_scale_delta,c_n))
                vect_p_max=vect_p[:,np.argmax(vals_p)]
                orientation_emp=np.arctan(vect_p_max[1]/vect_p_max[0])                                     # orientation empirique du champ
                meanof_tild+=orientation_emp/nb_realst                                                     # Calcul de l'estimateur


    ################################################################ Regression linéaire ##################################################################
        else:
            for i,l in enumerate(scales):
                vals_p,vect_p=np.linalg.eig(TSORE(H,delta,alpha,index,l,c_n))
                log2_sumvalp_ic[i,:]=np.r_[np.log2(vals_p.sum()),(vals_p.max()-vals_p.min())/vals_p.sum()]
            
            # Prepation des donnees pour la regression
            x=scales.reshape(-1,1)#np.arange(len(y)).reshape(-1,1)
            x_train,x_test,y_train,y_test=train_test_split(x,log2_sumvalp_ic[:,0],train_size=0.75,shuffle=False)
            scoring='neg_root_mean_squared_error'# Choix de la mesure de performance

            # Entrainnement et ajustement du model
            Model=LinearRegression()
            result=cross_validate(Model,x_train,y_train,scoring=scoring,cv=cv,return_estimator=True)

            # Resultats des performances du model suivant les plits
            r=result['test_score'].round(3);n=np.argwhere(r==r.max())[0][0]
            p=result['estimator'][n]
            tild_H=p.coef_[0]/2-1 #estimateur de H
            tild_c=np.exp2(p.intercept_)#/tild_c_phi # estimateur de l'intercept
            ic_at_best_scale=log2_sumvalp_ic[:,1][best_scale_delta]

            # Calcul des estimateurs
            meanof_tild_H+=tild_H/nb_realst
            meanof_tild_ic+=ic_at_best_scale/nb_realst
            meanof_tild=meanof_tild_H,meanof_tild_ic

    if alpha!=None:
        if delta>5*pi/16:
            print("L'orientation moyenne de la texture est:"+str(np.round(meanof_tild,3))+". Attention, ce résultat n'est pas garantie..!",'\n')
        elif alpha==-pi/2:
            print("Désolé, nous ne sommes pas en mesure d'estimer cette orientation..!",'\n')
        else:
            print("L'orientation moyenne de la texture est:"+str(np.round(meanof_tild,3)))
    else:
        if (H==0.99 or delta/H<=(pi/64)/0.25):
            print("L'autosimilarité moyen de la texture est:"+str(np.round(meanof_tild[0],3))+". Attention, ce résultat n'est pas garantie..!",'\n')
        else:
            print("L'autosimilarité moyen de la texture est:"+str(np.round(meanof_tild[0],3))+'\n')

        if H<=0.35 or H>=0.85:
            print("L'indice d'anisotropie moyen de la texture est:"+str(np.round(meanof_tild[1],3))+". Attention, ce résultat n'est pas garantie..!",'\n')
        else:
            print("L'indice d'anisotropie moyen de la texture est:"+str(np.round(meanof_tild[1],3))+'\n')
    #return meanof_tild

In [93]:
H_delta_alpha_estimator(H=0.5,delta=pi/8,alpha=-pi/6)

L'orientation moyenne de la texture est:-0.515
